## Here I will take the key features from my data set and run them through weighted classificaton models, current models are Random Forest and Logit

due to class imbalance and non-linear features my models are currently correctly identifying approx 10- 50% of the failed hard drives and mis attributing approx 5 - 25% of the working drives as broken. I will implement non-linear feature treatment and svm to address this in the coming days.

In [2]:
import pandas as pd
import sklearn
import seaborn as sns

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn import svm, ensemble, linear_model, neighbors, naive_bayes

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score

In [19]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, 
                              AdaBoostClassifier, BaggingRegressor)
from sklearn import linear_model, svm, metrics, ensemble

In [7]:
from sklearn import model_selection, preprocessing

In [8]:
df = pd.read_pickle("2016_pickle_norm.pkl")

In [9]:
df.failure.value_counts()

0    1013362
1      62707
Name: failure, dtype: int64

In [10]:
df.head()

,date,serial_number,model,capacity_bytes,failure,smart_187_normalized,smart_188_normalized,smart_189_normalized,smart_192_normalized,smart_197_normalized,smart_198_normalized,smart_242_normalized
0,2016-01-01,W300K1XB,ST4000DM000,4000787030016,0,100.0,100.0,96.0,100.0,100.0,100.0,100.0
1,2016-01-01,Z300H1WJ,ST4000DM000,4000787030016,0,100.0,100.0,97.0,100.0,100.0,100.0,100.0
2,2016-01-01,Z304JL7B,ST4000DM000,4000787030016,0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
3,2016-01-01,Z302BVHP,ST4000DM000,4000787030016,0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
4,2016-01-01,S300Z5BS,ST4000DM000,4000787030016,0,100.0,100.0,100.0,100.0,100.0,100.0,100.0


In [11]:
df.count()

date                    1076069
serial_number           1076069
model                   1076069
capacity_bytes          1076069
failure                 1076069
smart_187_normalized    1076069
smart_188_normalized    1076069
smart_189_normalized    1076069
smart_192_normalized    1076069
smart_197_normalized    1076069
smart_198_normalized    1076069
smart_242_normalized    1076069
dtype: int64

In [54]:
y,X= df['failure'],df.drop(['failure','serial_number','model','capacity_bytes','date'],axis=1)


In [55]:
x_train, x_testval, y_train, y_testval = model_selection.train_test_split(X, y, test_size=.5, stratify=y)

x_val, x_test, y_val, y_test = model_selection.train_test_split(x_testval, y_testval, test_size=.5, stratify=y_testval)

In [56]:
x_test.shape

(269018, 7)

In [57]:
#scaler = preprocessing.MinMaxScaler()
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)

x_val = scaler.transform(x_val)

x_test = scaler.transform(x_test)

In [58]:
print(len(x_train), len(x_val), len(x_test))

538034 269017 269018


In [107]:
#X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2,random_state=13)

#X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=43)

In [20]:
def print_metrics(model):
  
    y_pred = model.predict(x_val)

    print(metrics.classification_report(y_val, y_pred))

In [59]:
#logit=linear_model.LogisticRegression(C = 0.1,class_weight="balanced", solver='lbfgs')
logit=linear_model.LogisticRegression(C = 0.1,class_weight=class_weights, solver='lbfgs')
logit.fit(x_train, y_train)

print_metrics(logit)

              precision    recall  f1-score   support

           0       0.95      0.99      0.97    253340
           1       0.33      0.10      0.15     15677

    accuracy                           0.94    269017
   macro avg       0.64      0.54      0.56    269017
weighted avg       0.91      0.94      0.92    269017



In [83]:
class_weights = {
      0: 1,
      1: 13}

In [92]:
RandomForest_weights = RandomForestClassifier(class_weight=class_weights,n_estimators=200)
RandomForest_weights.fit(x_train, y_train)

print_metrics(RandomForest_weights)

              precision    recall  f1-score   support

           0       0.96      0.81      0.88    253340
           1       0.12      0.42      0.19     15677

    accuracy                           0.79    269017
   macro avg       0.54      0.62      0.53    269017
weighted avg       0.91      0.79      0.84    269017



In [78]:
logit=linear_model.LogisticRegression(C = 0.9,class_weight=class_weights, solver='lbfgs')
#scaler = StandardScaler()

In [79]:
#scaler.fit(X_train)
#X_train_scale=scaler.transform(X_train)


In [84]:
logit.fit(x_train,y_train)

LogisticRegression(C=0.9, class_weight={0: 1, 1: 12}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [85]:
logit.score(x_train,y_train)

0.9302553370233108

In [86]:
confusion_matrix(y_test, logit.predict(x_test))

array([[248689,   4652],
       [ 14163,   1514]])

In [88]:
randomforest = RandomForestClassifier(class_weight=class_weights,n_estimators=200)

In [89]:
randomforest.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 13},
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=200, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [90]:
randomforest.score(x_train,y_train)

0.7851604173713929

In [91]:
confusion_matrix(y_test, randomforest.predict(x_test))

array([[204535,  48806],
       [  9231,   6446]])